In [3]:
import numpy as np 
import matplotlib.pyplot as plt

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
palette = sns.color_palette('bright')
palette

[(0.00784313725490196, 0.24313725490196078, 1.0),
 (1.0, 0.48627450980392156, 0.0),
 (0.10196078431372549, 0.788235294117647, 0.2196078431372549),
 (0.9098039215686274, 0.0, 0.043137254901960784),
 (0.5450980392156862, 0.16862745098039217, 0.8862745098039215),
 (0.6235294117647059, 0.2823529411764706, 0.0),
 (0.9450980392156862, 0.2980392156862745, 0.7568627450980392),
 (0.6392156862745098, 0.6392156862745098, 0.6392156862745098),
 (1.0, 0.7686274509803922, 0.0),
 (0.0, 0.8431372549019608, 1.0)]

In [ ]:
#RK4 METHOD 

In [77]:
class Planet:
    def __init__ (self,PlanetName):
        self.PlanetName = PlanetName
        self.position = self._position() #Nowstate position
        self.velocity = self._velocity() #Nowstate velocity
        self.mu = self._mu()
        self.distance = self._distance() #Distance from the Sun 
        _AU = 1.496e+11                    #1AU to m 
        _hour = 3600                       #hour to second
        _ss = 7.71604938e-8                #sec^2 to hour^2
    
class PlanetMercury(Planet):
    def _distance(self):
        return float(1.496e+11*0.39)       #aphelion position
    def _position(self):
        return float(1.496e+11*0.39)       #initial position
    def _velocity(self):
        return 47.4e3
    def _mu(self):                        #Mu = G*M [m^3/s^2]
        return 2.2032e13

class PlanetVenus(Planet):
    def _distance(self):
        return float(1.496e+11*0.723)
    def _position(self):
        return float(1.496e+11*0.723)
    def _velocity(self):
        return 35.1e3
    def _mu(self):
        return 3.24859e14

class PlanetEarth(Planet):
    def _distance(self):
        return float(1.496e+11*1)
    def _position(self):
        return float(1.496e+11*1)
    def _velocity(self):
        return 29.8e3
    def _mu(self):
        return 3.986004418e14

class PlanetMars(Planet):
    def _distance(self):
        return float(1.496e+11*1.524)
    def _position(self):
        return float(1.496e+11*1.524)
    def _velocity(self):
        return 24.1e3
    def _mu(self):
        return 4.282837e13

class PlanetJupiter(Planet):
    def _distance(self):
        return float(1.496e+11*5.203)
    def _position(self):
        return float(1.496e+11*5.203)
    def _velocity(self):
        return 13.1e3
    def _mu(self):
        return 1.26686534e17

class PlanetSaturn(Planet):
    def _distance(self):
        return float(1.496e+11*9.539)
    def _position(self):
        return float(1.496e+11*9.539)
    def _velocity(self):
        return 9.7e3
    def _mu(self):
        return 3.7931187e16

class PlanetUranus(Planet):
    def _distance(self):
        return float(1.496e+11*19.18)
    def _position(self):
        return float(1.496e+11*19.18)
    def _velocity(self):
        return 6.8e3
    def _mu(self):
        return 5.793939e15

class PlanetNeptune(Planet):
    def _distance(self):
        return float(1.496e+11*30.06)
    def _position(self):
        return float(1.496e+11*30.06)
    def _velocity(self):
        return 5.4e3
    def _mu(self):
        return 6.836529e15

class PlanetPluto(Planet):
    def _distance(self):
        return float(1.496e+11*39.53)
    def _position(self):
        return float(1.496e+11*39.53)
    def _velocity(self):
        return 4.7e3
    def _mu(self):
        return 8.71e11

class CometHalley(Planet):
    def _distance(self):
        return float(1.496e+11*35.3)
    def _position(self):
        return float(1.496e+11*35.3)
    def _velocity(self):
        return 1000
    def _mu(self):
        return 6.670e3*2.2

In [79]:
def planets(PlanetName):
    if PlanetName == 1:
        return PlanetMercury('Mercury')
    elif PlanetName == 2:
        return PlanetVenus('Venus')
    elif PlanetName == 3:
        return PlanetEarth('Earth')
    elif PlanetName == 4:
        return PlanetMars('Mars')
    elif PlanetName == 5:
        return PlanetJupiter('Jupiter')
    elif PlanetName == 6:
        return PlanetSaturn('Saturn')
    elif PlanetName == 7:
        return PlanetUranus('Uranus')
    elif PlanetName == 8:
        return PlanetNeptune('Neptune')
    elif PlanetName == 9:
        return PlanetPluto('Pluto')
    elif PlanetName == 10:
        return CometHalley('Halley')
    else:
        raise Exception("There is no such planet in the solar system.")
        

In [88]:
class SolarSystem(list):
    def __init__(self):
        self.planets = [planets(i) for i in range(1,11)]


In [89]:
Solsy = SolarSystem()

In [97]:
Solsy.planets